# Redis Basics for Subscription Tracking

This notebook demonstrates how to use Redis for tracking market data subscriptions, with step-by-step examples.

In [1]:
# Required imports
import redis
import json
from datetime import datetime
import time
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

## 1. Connecting to Redis

Let's start by connecting to our Redis instance in the Docker container.

In [ ]:
# Connect to Redis
redis_client = redis.Redis(
    host='redis',      # Using container name in Docker Compose network
    port=6379,         # Default Redis port
    decode_responses=True  # This returns strings instead of bytes
)

# Test the connection
try:
    response = redis_client.ping()
    print(f"Connected to Redis: {response}")
except Exception as e:
    print(f"Error connecting to Redis: {e}")
    
# Show Redis info
info = redis_client.info()
print(f"Redis version: {info['redis_version']}")
print(f"Connected clients: {info['connected_clients']}")

## 2. Basic Redis Data Types

Let's explore the basic Redis data types we'll use for subscription tracking.

In [3]:
# Clear any previous test data (careful with this in production!)
for key in redis_client.keys("test:*"):
    redis_client.delete(key)

In [ ]:
# STRING: Simple key-value pairs
redis_client.set("test:string:symbol", "SPX")
print(f"String value: {redis_client.get('test:string:symbol')}")

In [ ]:
# HASH: Object with fields (perfect for subscription data)
redis_client.hset( 
    "test:hash:subscription:SPX", 
    mapping={
        "symbol": "SPX", 
        "active": "true",
        "subscribeTime": datetime.now().isoformat()
    }
)
print(f"Hash value: {redis_client.hgetall('test:hash:subscription:SPX')}")

In [ ]:
# SET: Collection of unique values (good for tracking active subscriptions)
redis_client.sadd("test:set:active_subscriptions", "SPX", "AAPL", "QQQ")
print(f"Set members: {redis_client.smembers('test:set:active_subscriptions')}")

In [ ]:
# ZSET: Sorted set by score (good for time-based tracking)
now = time.time()

# Redis CLI equivalent:
# > ZADD test:zset:subscription_activity 1718557800 SPX 1718554200 AAPL 1718557500 QQQ
# Note: In Redis CLI, you need to use actual numeric timestamps, not placeholders

redis_client.zadd("test:zset:subscription_activity", {
    "SPX": now,
    "AAPL": now - 3600,  # 1 hour ago
    "QQQ": now - 300     # 5 minutes ago
})

# > ZRANGE test:zset:subscription_activity 0 -1 WITHSCORES
print(f"Sorted set range: {redis_client.zrange('test:zset:subscription_activity', 0, -1, withscores=True)}")

# LIST: Ordered collection of elements (good for message queues or history)
redis_client.rpush("test:list:price_updates", "SPX:4500", "SPX:4505", "SPX:4510")
print(f"List values: {redis_client.lrange('test:list:price_updates', 0, -1)}")

# Removing individual elements from different data structures

# 1. Remove from LIST
# Redis CLI: LREM test:list:price_updates 1 "SPX:4505"
redis_client.lrem("test:list:price_updates", 1, "SPX:4505")
print(f"List after removal: {redis_client.lrange('test:list:price_updates', 0, -1)}")

# 2. Remove from SET
# Redis CLI: SREM test:set:active_subscriptions "AAPL"
redis_client.srem("test:set:active_subscriptions", "AAPL")
print(f"Set after removal: {redis_client.smembers('test:set:active_subscriptions')}")

# 3. Remove from HASH (delete a field)
# Redis CLI: HDEL test:hash:subscription:SPX active
redis_client.hdel("test:hash:subscription:SPX", "active")
print(f"Hash after field removal: {redis_client.hgetall('test:hash:subscription:SPX')}")

# 4. Remove from ZSET
# Redis CLI: ZREM test:zset:subscription_activity "QQQ"
redis_client.zrem("test:zset:subscription_activity", "QQQ")
print(f"Sorted set after removal: {redis_client.zrange('test:zset:subscription_activity', 0, -1, withscores=True)}")

# Accessing individual elements
# 1. Get specific LIST element by index
# Redis CLI: LINDEX test:list:price_updates 0
first_price = redis_client.lindex("test:list:price_updates", 0)
print(f"First price update: {first_price}")

# 2. Check if element exists in SET
# Redis CLI: SISMEMBER test:set:active_subscriptions "SPX"
is_spx_active = redis_client.sismember("test:set:active_subscriptions", "SPX")
print(f"Is SPX active? {is_spx_active}")

# 3. Get specific HASH field
# Redis CLI: HGET test:hash:subscription:SPX symbol
symbol = redis_client.hget("test:hash:subscription:SPX", "symbol")
print(f"Symbol from hash: {symbol}")

# 4. Get score of element in ZSET
# Redis CLI: ZSCORE test:zset:subscription_activity "SPX"
spx_score = redis_client.zscore("test:zset:subscription_activity", "SPX")
print(f"SPX activity timestamp: {spx_score}")


## 3. Creating Subscription Entries

Now let's create more realistic subscription entries that mimic what we'd use in the TastyTrade SDK.

In [ ]:
def create_simple_subscription(symbol, channel_type, interval=None):
    """Create a simple subscription entry in Redis."""
    timestamp = datetime.now().isoformat()
    
    # Create a key that includes relevant information
    if interval:
        key = f"sub:{symbol}:{interval}"
    else:
        key = f"sub:{symbol}:{channel_type}"
    
    # Store the subscription data as a hash
    redis_client.hset(key, mapping={
        "symbol": symbol,
        "channel_type": channel_type,
        "interval": interval or "",
        "created_at": timestamp,
        "updated_at": timestamp,
        "active": "true"
    })
    
    # Add to the appropriate sets
    redis_client.sadd(f"active:{channel_type}", key)
    redis_client.sadd("active:all", key)
    
    # Record the activity time
    redis_client.zadd("activity:times", {key: time.time()})
    
    return key

# Create various subscription types
create_simple_subscription("SPX", "quote")
create_simple_subscription("SPX", "candle", "5m")
create_simple_subscription("SPX", "candle", "1h")
create_simple_subscription("AAPL", "quote")
create_simple_subscription("QQQ", "quote")
create_simple_subscription("BTC/USD:CXTALP", "candle", "15m")

# List all our subscriptions
print("All active subscriptions:")
for key in redis_client.smembers("active:all"):
    print(f"- {key}: {redis_client.hgetall(key)}")

## 4. Updating Subscription Status

Now let's simulate updating a subscription with market data.

In [ ]:
def update_subscription(sub_key, data=None):
    """Update a subscription with market data."""
    if not redis_client.exists(sub_key):
        print(f"Subscription {sub_key} does not exist")
        return False
    
    # Update the last update time
    redis_client.hset(sub_key, "updated_at", datetime.now().isoformat())
    
    # Update the market data if provided
    if data:
        # Store latest data as JSON string
        redis_client.hset(sub_key, "last_data", json.dumps(data))
    
    # Update the activity timestamp
    redis_client.zadd("activity:times", {sub_key: time.time()})
    
    return True

# Update some subscriptions
update_subscription("sub:SPX:quote", {"bid": 5120.25, "ask": 5120.50})
update_subscription("sub:SPX:5m", {"open": 5118.75, "high": 5125.50, "low": 5118.50, "close": 5120.25})

# Show the updated data
print("Updated SPX quote:")
print(redis_client.hgetall("sub:SPX:quote"))

# Parse the JSON data
last_data = json.loads(redis_client.hget("sub:SPX:quote", "last_data"))
print(f"Parsed market data: {last_data}")

## 5. Querying Subscriptions

Let's implement some useful queries for subscription data.

In [ ]:
def get_all_subscriptions():
    """Get all subscriptions as a pandas DataFrame."""
    all_subs = []
    
    for key in redis_client.smembers("active:all"):
        data = redis_client.hgetall(key)
        
        # Parse JSON data if present
        if "last_data" in data:
            try:
                data["last_data"] = json.loads(data["last_data"])
            except json.JSONDecodeError:
                pass
                
        # Add to our list
        all_subs.append(data)
    
    return pd.DataFrame(all_subs)

def find_stale_subscriptions(minutes=5):
    """Find subscriptions not updated in the last N minutes."""
    cutoff = time.time() - (minutes * 60)
    stale_keys = redis_client.zrangebyscore("activity:times", 0, cutoff)
    
    stale_subs = []
    for key in stale_keys:
        if redis_client.sismember("active:all", key):
            stale_subs.append(redis_client.hgetall(key))
    
    return pd.DataFrame(stale_subs)

# Display all subscriptions
df = get_all_subscriptions()
display(df)

# Make one subscription stale for demonstration
redis_client.zadd("activity:times", {"sub:QQQ:quote": time.time() - 600})  # 10 minutes ago

# Find stale subscriptions (older than 5 minutes)
stale_df = find_stale_subscriptions(minutes=5)
print("Stale subscriptions:")
display(stale_df)

## 6. Working with Subscription Stats

Let's track some basic stats about our subscriptions.

In [ ]:
def get_subscription_stats():
    """Get basic stats about subscriptions."""
    stats = {
        "total": len(redis_client.smembers("active:all")),
        "by_channel": {},
        "by_symbol": {}
    }
    
    # Count by channel type
    for channel in ["quote", "trade", "candle", "greeks"]:
        stats["by_channel"][channel] = len(redis_client.smembers(f"active:{channel}"))
    
    # Count by symbol
    df = get_all_subscriptions()
    if not df.empty and "symbol" in df.columns:
        symbol_counts = df["symbol"].value_counts().to_dict()
        stats["by_symbol"] = symbol_counts
    
    return stats

# Get and display stats
stats = get_subscription_stats()
print(f"Total active subscriptions: {stats['total']}")
print(f"By channel type: {stats['by_channel']}")
print(f"By symbol: {stats['by_symbol']}")

# Create a simple visualization
plt.figure(figsize=(10, 6))

# Plot channel counts
plt.subplot(1, 2, 1)
plt.bar(stats['by_channel'].keys(), stats['by_channel'].values())
plt.title('Subscriptions by Channel')
plt.xlabel('Channel Type')
plt.ylabel('Count')

# Plot symbol counts
plt.subplot(1, 2, 2)
plt.bar(stats['by_symbol'].keys(), stats['by_symbol'].values())
plt.title('Subscriptions by Symbol')
plt.xlabel('Symbol')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

## 7. Implementing State Changes

Let's implement activation/deactivation of subscriptions.

In [ ]:
def deactivate_subscription(sub_key):
    """Deactivate a subscription."""
    if not redis_client.exists(sub_key):
        return False
    
    # Mark as inactive
    redis_client.hset(sub_key, "active", "false")
    redis_client.hset(sub_key, "deactivated_at", datetime.now().isoformat())
    
    # Remove from active sets
    redis_client.srem("active:all", sub_key)
    
    # Get channel type and remove from that set
    channel_type = redis_client.hget(sub_key, "channel_type")
    if channel_type:
        redis_client.srem(f"active:{channel_type}", sub_key)
    
    # Add to inactive set
    redis_client.sadd("inactive:all", sub_key)
    
    return True

def reactivate_subscription(sub_key):
    """Reactivate an inactive subscription."""
    if not redis_client.exists(sub_key):
        return False
    
    # Mark as active
    redis_client.hset(sub_key, "active", "true")
    redis_client.hset(sub_key, "reactivated_at", datetime.now().isoformat())
    
    # Add to active sets
    redis_client.sadd("active:all", sub_key)
    
    # Get channel type and add to that set
    channel_type = redis_client.hget(sub_key, "channel_type")
    if channel_type:
        redis_client.sadd(f"active:{channel_type}", sub_key)
    
    # Remove from inactive set
    redis_client.srem("inactive:all", sub_key)
    
    # Update activity time
    redis_client.zadd("activity:times", {sub_key: time.time()})
    
    return True

# Deactivate a subscription
print(f"Before: {redis_client.hgetall('sub:AAPL:quote')}")
deactivate_subscription("sub:AAPL:quote")
print(f"After deactivation: {redis_client.hgetall('sub:AAPL:quote')}")

# Reactivate it
reactivate_subscription("sub:AAPL:quote")
print(f"After reactivation: {redis_client.hgetall('sub:AAPL:quote')}")

## 8. Simulating TastyTrade Subscription Updates

Let's simulate data updates coming from the TastyTrade WebSocket.

In [ ]:
import random
from IPython.display import clear_output

def simulate_market_update(symbol, channel_type, interval=None):
    """Simulate a market data update for a symbol."""
    # Construct the subscription key
    if interval:
        sub_key = f"sub:{symbol}:{interval}"
    else:
        sub_key = f"sub:{symbol}:{channel_type}"
    
    # Different data based on channel type
    if channel_type == "quote":
        # Generate quote data
        base_price = 100.0 if symbol == "AAPL" else 400.0 if symbol == "QQQ" else 5000.0
        price_change = random.uniform(-2.0, 2.0)
        mid_price = base_price + price_change
        bid = mid_price - random.uniform(0.01, 0.1)
        ask = mid_price + random.uniform(0.01, 0.1)
        data = {
            "bid": round(bid, 2),
            "ask": round(ask, 2),
            "last": round(mid_price, 2),
            "volume": int(random.uniform(100, 10000)),
            "timestamp": datetime.now().isoformat()
        }
    elif channel_type == "candle":
        # Generate candle data
        base_price = 100.0 if symbol == "AAPL" else 400.0 if symbol == "QQQ" else 5000.0
        movement = random.uniform(-5.0, 5.0)
        mid_price = base_price + movement
        
        # Create a realistic candle
        open_price = mid_price - random.uniform(-2.0, 2.0)
        close_price = mid_price 
        high_price = max(open_price, close_price) + random.uniform(0.1, 1.0)
        low_price = min(open_price, close_price) - random.uniform(0.1, 1.0)
        data = {
            "open": round(open_price, 2),
            "high": round(high_price, 2),
            "low": round(low_price, 2),
            "close": round(close_price, 2),
            "volume": int(random.uniform(100, 10000)),
            "timestamp": datetime.now().isoformat()
        }
    else:
        # Generic data
        data = {"timestamp": datetime.now().isoformat()}
    
    # Update the subscription
    update_subscription(sub_key, data)
    return data

def run_simulation(iterations=10, delay=1):
    """Run a simulation of market updates."""
    subscriptions = [
        ("SPX", "quote", None),
        ("SPX", "candle", "5m"),
        ("AAPL", "quote", None),
        ("QQQ", "quote", None) 
    ]
    
    for i in range(iterations):
        clear_output(wait=True)
        print(f"Simulation iteration {i+1}/{iterations}\n")
        
        # Update a random selection of subscriptions
        for _ in range(random.randint(1, len(subscriptions))):
            symbol, channel_type, interval = random.choice(subscriptions)
            data = simulate_market_update(symbol, channel_type, interval)
            print(f"Updated {symbol} {channel_type} {interval or ''}:")
            print(f"  {data}\n")
        
        # Show subscription stats
        stats = get_subscription_stats()
        print(f"Active subscriptions: {stats['total']}")
        
        # Find recent updates
        cutoff = time.time() - 5  # Last 5 seconds
        recent_keys = redis_client.zrangebyscore("activity:times", cutoff, "+inf")
        print(f"Recent updates (last 5s): {len(recent_keys)}")
        
        time.sleep(delay)

# Run a short simulation
run_simulation(iterations=5, delay=2)

## 9. Redis Pub/Sub for Subscription Events

Let's experiment with Redis Pub/Sub for real-time notifications.

In [ ]:
def publish_event(event_type, data):
    """Publish an event to Redis."""
    payload = {
        "type": event_type,
        "timestamp": datetime.now().isoformat(),
        "data": data
    }
    return redis_client.publish("subscription:events", json.dumps(payload))

# Publish some events
publish_event("subscription:created", {"symbol": "SPX", "channel": "quote"})
publish_event("subscription:updated", {"symbol": "SPX", "channel": "quote", "bid": 5120.25})
publish_event("subscription:deactivated", {"symbol": "AAPL", "channel": "quote"})

## 10. Exercise: Implement Subscription Batch Operations

As a hands-on exercise, try implementing a function to batch-create multiple subscriptions.

In [ ]:
# TODO: Implement a batch subscription function
def batch_create_subscriptions(subscriptions):
    """Create multiple subscriptions in a single batch.
    
    Args:
        subscriptions: List of (symbol, channel_type, interval) tuples
        
    Returns:
        List of created subscription keys
    """
    # Your implementation here
    pass

# Example usage once implemented:
# batch = [
#     ("AMZN", "quote", None),
#     ("AMZN", "candle", "5m"),
#     ("MSFT", "quote", None),
#     ("GOOG", "quote", None)
# ]
# created_keys = batch_create_subscriptions(batch)
# print(f"Created {len(created_keys)} subscriptions")

## 11. Exercise: Implement Subscription Search

Implement a function to search for subscriptions by symbol or other criteria.

In [ ]:
# TODO: Implement a subscription search function
def search_subscriptions(symbol=None, channel_type=None, active_only=True):
    """Search for subscriptions matching criteria.
    
    Args:
        symbol: Filter by symbol (optional)
        channel_type: Filter by channel type (optional) 
        active_only: Whether to return only active subscriptions
        
    Returns:
        DataFrame of matching subscriptions
    """
    # Your implementation here
    pass

# Example usage once implemented:
# spx_subs = search_subscriptions(symbol="SPX")
# display(spx_subs)

## 12. Exercise: Implement Redis Pipeline for Bulk Operations

Learn how to use Redis pipelines for more efficient bulk operations.

In [ ]:
# TODO: Implement bulk operations using Redis pipeline
def bulk_update_subscriptions(subscription_data):
    """Update multiple subscriptions in a single operation.
    
    Args:
        subscription_data: Dict mapping subscription keys to update data
        
    Returns:
        Number of updated subscriptions
    """
    # Your implementation using pipeline
    pass

# Example usage once implemented:
# updates = {
#     "sub:SPX:quote": {"last_price": 5125.50},
#     "sub:AAPL:quote": {"last_price": 185.25},
#     "sub:QQQ:quote": {"last_price": 425.80}
# }
# updated_count = bulk_update_subscriptions(updates)
# print(f"Updated {updated_count} subscriptions")

## 13. Cleaning Up (Optional)

If you want to clean up your test data, you can use this cell.

In [ ]:
# WARNING: This will delete all subscription data
# Uncomment to run

# def cleanup_subscription_data():
#     """Remove all subscription data from Redis."""
#     # Delete all subscription keys
#     for key in r.keys("sub:*"):
#         r.delete(key)
    
#     # Delete all set keys
#     for key in r.keys("active:*") + r.keys("inactive:*"):
#         r.delete(key)
    
#     # Delete activity tracking
#     r.delete("activity:times")
    
#     print("All subscription data cleaned up")
    
# cleanup_subscription_data()